In [22]:
import json 

In [23]:
import copy

### read the initial json

In [24]:
path="/home/ids/gbrison/FC/fc-clip/datasets/cityscapes/gtFine/cityscapes_panoptic_train.json"
with open(path, 'r') as file:
    data = json.load(file)


# get the ids that can be changed 

In [25]:
lis_id_class=[]
for i in data['annotations']:
    for k in i['segments_info']:
        if k['category_id'] < 50:
            if k['category_id'] not in lis_id_class:
                lis_id_class.append(k['category_id'])

In [26]:
lis_id_class

[7, 8, 11, 17, 20, 21, 22, 23, 24, 25, 26, 33, 12, 13, 19, 32, 28, 27, 31]

# Read the output json after the inference 


In [27]:
path_results="/tsi/hi-paris/FCCLIP_results/inference_results/fcclip_cocopan_train/inference/predictions.json"
with open(path_results, 'r') as file:
    outputs = json.load(file)


# There are objects that are not classified with the model 
Proof: For the first image, we have 21 objects in the input annotation and the model detects 12 objects.

In [28]:
len(data['annotations'][0]['segments_info']),len(outputs['annotations'][0]['segments_info'])

(21, 12)

# Get  classes that represent isthing

In [29]:
isthing_classes=[]
for i in data["categories"]: 
    if i['isthing']==1:
        isthing_classes.append(i['id'])

In [30]:
isthing_classes

[24, 25, 26, 27, 28, 31, 32, 33]

#  Read data for intermediate results to find correspondences between labels.

In [31]:
import pickle

In [32]:
with open("/tsi/hi-paris/FCCLIP_results/inference_results/train/aal", "rb") as fp:   # Unpickling
     gt_pred_map = pickle.load(fp)
with open("/tsi/hi-paris/FCCLIP_results/inference_results/train/all1", "rb") as fp:   # Unpickling
     gt_ann = pickle.load(fp)
with open("/tsi/hi-paris/FCCLIP_results/inference_results/train/all2", "rb") as fp:   # Unpickling
     pred_ann = pickle.load(fp)

In [33]:
with open("/tsi/hi-paris/FCCLIP_results/inference_results/train/pred_segms_list", "rb") as fp:   # Unpickling
     pred_segms_list = pickle.load(fp)
with open("/tsi/hi-paris/FCCLIP_results/inference_results/train/gt_segms_list", "rb") as fp:   # Unpickling
     gt_segms_list = pickle.load(fp)

# Found the matching

In [34]:
def test_(data, id, val):
    for i in data:
        if i[0]==id[0]:
            return True, i
    return False, None

def get_matchin(pred_segms_list,gt_segms_list,matchin_list):
    matchin_ids_data=[]
    for pred_segms, gt_segms, matchin in zip(pred_segms_list, gt_segms_list, matchin_list):
        
        lis={}
        for label_tuple, intersection in matchin.items():
            gt_label, pred_label = label_tuple
            if gt_label not in gt_segms:
                continue
            if pred_label not in pred_segms:
                continue
            if gt_segms[gt_label]['iscrowd'] == 1:
                continue
            if gt_segms[gt_label]['category_id'] != pred_segms[pred_label]['category_id']:
                continue
            if label_tuple in lis:
                if lis[label_tuple] < intersection :
                    lis[label_tuple]=intersection
            elif label_tuple not in lis:
                test,id=test_(lis, label_tuple, intersection)
                if test:
                    if lis[id] < intersection:
                        lis[id]=intersection   
                else:
                    lis[label_tuple]=intersection

        matchin_ids_data.append(lis)
    return matchin_ids_data
        

In [35]:
matchin_ids_data=get_matchin(pred_segms_list,gt_segms_list,gt_pred_map)

### exemple of the matching for the first image

In [36]:

# convert data matching
new_data_matchin=[]
for i in matchin_ids_data:
    lis={}
    for j in i:
        lis[j[0]]=j[1]
    new_data_matchin.append(lis) 

# Preparate jsons

In [39]:
def prepare_json(input_json, data_matchin,pred_segms_list, conf):
    path="/home/ids/gbrison/FC/fc-clip/datasets/cityscapes/gtFine/cityscapes_panoptic_train.json"
    with open(path, 'r') as file:
        new_json = json.load(file)

    list_of_labels= [7, 8, 11, 17, 20, 21, 22, 23, 24, 25, 26, 33, 12, 13, 19, 32, 28, 27, 31]
    in_vocab_labels=list_of_labels[:-conf]
    out_vocab_labels=list_of_labels[-conf:]
    for annot_new, outputs, matching in zip(new_json["annotations"],pred_segms_list, data_matchin):
        for segment in annot_new["segments_info"]:
            if segment['category_id'] in out_vocab_labels:
                if segment['id'] in matching:
                    segment['category_id']=outputs[matching[segment['id']]]["category_id"] 
    return new_json

    

new_json=prepare_json(data, new_data_matchin,pred_segms_list, 5)


In [40]:
save_json="/home/ids/gbrison/FC/fc-clip/datasets/cityscapes/gtFine/cityscapes_panoptic_train5.json"
with open(save_json, 'w') as f:
    json.dump(new_json, f)

In [20]:
la=0
for k in range(len(new_json["annotations"])):
    for i,j in zip(new_json["annotations"][k]["segments_info"],data["annotations"][k]["segments_info"]):
        if i['category_id']!=j['category_id']:
            la+=1
la

In [9]:
import torch
import torch.nn.functional as F

# Exemple de tenseur
input_tensor = torch.tensor([0.5, 0.2, 0.1, 0.0, 0.2])

# Appliquer le seuil
threshold_value = .2
thresholded_tensor = F.threshold(input_tensor, threshold_value, 0)

print(thresholded_tensor)

tensor([0.5000, 0.0000, 0.0000, 0.0000, 0.0000])
